In [1]:
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt
##Classe com vários métodos de pre-processamento de texto em português criado pelo grupo F03
import utils.preprocessing_portuguese as preprossPT

In [2]:
def limpeza_texto(page_text):
    txt_process = preprossPT.TextPreProcessing()
    
    page_text = page_text.lower()
    
    page_text = txt_process.remove_pronouns(page_text)

    page_text = txt_process.remove_reduced_or_contracted_words(page_text)

    page_text = txt_process.remove_adverbs(page_text)

    page_text = txt_process.remove_special_characters(page_text)

    page_text = txt_process.remove_excessive_spaces(page_text)

    page_text = txt_process.remove_accents(page_text)

    page_text = txt_process.remove_stopwords(page_text)

    page_text = txt_process.remove_symbols_from_numbers(page_text)

    page_text = txt_process.remove_numbers(page_text)

    page_text = txt_process.remove_urls(page_text)
    
    page_text = txt_process.remove_person_names(page_text)
    
    #Removendo letras sozinhas no texto
    page_text = re.sub(r'(?:^| )\w(?:$| )', ' ', page_text).strip()

    page_text = page_text.replace("_","")
    
    return page_text

In [3]:
def cities_prep(df):
    return df['cities'].str.replace("'","").str.replace(" ",",").str.replace(",,",",").str.replace(",",", ").str.replace("\n","").str.replace("[","").str.replace("]","").str.lower()
def flatten(t):
    return list(set([item.replace(" ","") for sublist in t for item in sublist]))

### Gera Classes_v2

In [16]:
sheet_names = ['sao-bento-abade', 'olaria', 'coqueiral', 'cristais', 'pirapetinga', 'passa-vinte', 'arantina', 'itamarati', 'ribeirao-vermelho', 'ijaci']
file_dir = 'resultado_m03_meta_classes_extraction/resultado_final.xlsx'
all_sheets = []
for sheet in sheet_names:
    df = pd.read_excel(file_dir, sheet_name=sheet)
    df.columns = df.columns.str.lower()
    all_sheets.append(df)
# Concatena todos dataframes em apenas um
df_rotulos = pd.concat(all_sheets)
# Dropa linhas vazias e reseta o indice
#df_rotulos.dropna(how='all', subset=['doc_id', 'title', 'city', 'all_matches', 'meta-class',
#       'real_meta-class', 'real_class', 'key_words', 'obs', 'escaneado'], inplace=True)
df_rotulos.reset_index(drop=True, inplace=True)
df_rotulos.head()

/dados01/workspace/ufmg.f01dcc/py37/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,number,doc_id,title,city,all_matches,meta-class,real_meta-class,real_class,key_words,obs,...,unnamed: 16,unnamed: 17,unnamed: 18,unnamed: 19,unnamed: 20,unnamed: 21,unnamed: 22,unnamed: 23,unnamed: 24,unnamed: 25
0,0.0,275acd244d5791013bd8f4b3243a4a1a,prefeitura municipal de são bento abade mg adm...,sao-bento-abade,"['ata', 'ata']",ATA,ATA,ATA DE REGISTRO DE PREÇOS,"ATA, ATA DE REGISTRO DE PREÇOS",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2309298b1878b216dca666aba446a4cd,prefeitura municipal de são bento abade mg ...,sao-bento-abade,"['ata', 'sessão pública', 'edital']",ATA,ATA,ATA DE SESSÃO PÚBLICA,"ATA, tomada de preco",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,85c1a53d38e9f0128f437554523bf87c,prefeitura municipal de são bento abade estado...,sao-bento-abade,"['ata', 'ata', 'homologação', 'homologação', '...",ATA,ATA,ATA REFERENTE A FASE DE JULGAMENTO DAS EMPRESAS,"ATA, FASE DE JULGAMENTO",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,4d6035bf3212568e336fe0ab7d9fb215,prefeitura municipal de são bento abade estad...,sao-bento-abade,"['ata', 'sessão pública', 'edital']",ATA,ATA,ATA DO PREGÃO PRESENCIAL,"ATA, ATA DO PREGÃO PRESENCIAL",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,580e6393365c2957e221de56844dd011,prefeitura municipal de são bento abade estado...,sao-bento-abade,"['ata', 'sessão pública', 'edital', 'edital']",ATA,ATA,ATA DO PREGÃO PRESENCIAL,"sessao, publica",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
file_dir = 'resultado_m03_meta_classes_extraction/resultado_final.xlsx'
df_classes = pd.read_excel(file_dir, sheet_name="Classes")
df_classes = df_classes.dropna(subset=['final_class']).reset_index(drop=True)
df_classes.loc[df_classes['final_class']=='impulgnacao', 'final_class'] = 'impugnacao'
df_classes['cities'] = cities_prep(df_classes)
df_classes.loc[df_classes['cities'].isnull(), ['cities', 'num_cities']] = ['pirapetinga', 1]

final_class = df_classes['final_class'].tolist()
df_classes['final_class'] = [limpeza_texto(class_name) for class_name in final_class]

df_classes


df_rotulos = df_rotulos.dropna(subset=["real_class", "real_meta-class"], how="any")

display(df_rotulos.head())


/dados01/workspace/ufmg.f01dcc/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


,number,doc_id,title,city,all_matches,meta-class,real_meta-class,real_class,key_words,obs,...,unnamed: 16,unnamed: 17,unnamed: 18,unnamed: 19,unnamed: 20,unnamed: 21,unnamed: 22,unnamed: 23,unnamed: 24,unnamed: 25
0,0.0,275acd244d5791013bd8f4b3243a4a1a,prefeitura municipal de são bento abade mg adm...,sao-bento-abade,"['ata', 'ata']",ATA,ATA,ATA DE REGISTRO DE PREÇOS,"ATA, ATA DE REGISTRO DE PREÇOS",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2309298b1878b216dca666aba446a4cd,prefeitura municipal de são bento abade mg ...,sao-bento-abade,"['ata', 'sessão pública', 'edital']",ATA,ATA,ATA DE SESSÃO PÚBLICA,"ATA, tomada de preco",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,85c1a53d38e9f0128f437554523bf87c,prefeitura municipal de são bento abade estado...,sao-bento-abade,"['ata', 'ata', 'homologação', 'homologação', '...",ATA,ATA,ATA REFERENTE A FASE DE JULGAMENTO DAS EMPRESAS,"ATA, FASE DE JULGAMENTO",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,4d6035bf3212568e336fe0ab7d9fb215,prefeitura municipal de são bento abade estad...,sao-bento-abade,"['ata', 'sessão pública', 'edital']",ATA,ATA,ATA DO PREGÃO PRESENCIAL,"ATA, ATA DO PREGÃO PRESENCIAL",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,580e6393365c2957e221de56844dd011,prefeitura municipal de são bento abade estado...,sao-bento-abade,"['ata', 'sessão pública', 'edital', 'edital']",ATA,ATA,ATA DO PREGÃO PRESENCIAL,"sessao, publica",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
processed_classes = df_rotulos['real_class'].to_list()
#print(processed_classes) 
processed_classes = [limpeza_texto(class_name) for class_name in processed_classes]
'''for class_name in processed_classes:
    print(class_name)
    limpeza_texto(class_name)'''

df_rotulos['processed_classes'] = processed_classes

df_rotulos.loc[df_rotulos['real_meta-class'].isnull(), 'real_meta-class'] = 'NULL'

df_count_class_and_meta = df_rotulos.groupby(['real_meta-class', 'processed_classes']).size().reset_index(name='count').sort_values(by='count', ascending=False)
df_count_cities = df_rotulos.groupby(['real_meta-class', 'processed_classes']).agg({'city': lambda x: list(set(x))}).reset_index()
df_count_cities['num_cities'] = df_count_cities['city'].apply(lambda x: len(x))

df_count_class_and_meta = pd.merge(left=df_count_class_and_meta, right=df_count_cities, on=['real_meta-class', 'processed_classes'])

df_merge = pd.merge(left=df_count_class_and_meta, right=df_classes[['real_meta-class', 'final_meta-class', 'processed_classes', 'final_class', 'validate', 'obs','example']], 
         on=['real_meta-class', 'processed_classes'], how='left')
df_merge.rename({'city':'cities'}, inplace=True, axis=1)
df_merge = df_merge[['real_meta-class', 'final_meta-class', 'processed_classes', 'count', 'final_class', 'validate', 'obs', 'cities', 'num_cities', 'example']]
df_merge.to_csv('resultado_m03_meta_classes_extraction/aba_Classes_v2.csv',index=False)

# Sheet Classes
### Gera Anexo A

In [21]:
file_dir = 'resultado_m03_meta_classes_extraction/resultado_final.xlsx'
df_classes = pd.read_excel(file_dir, sheet_name="Classes_v2")
df_classes = df_classes.dropna(subset=['final_class']).reset_index(drop=True)
df_classes.loc[df_classes['final_class']=='impulgnacao', 'final_class'] = 'impugnacao'
df_classes['cities'] = cities_prep(df_classes)
df_classes.loc[df_classes['cities'].isnull(), ['cities', 'num_cities']] = ['pirapetinga', 1]

final_class = df_classes['final_class'].tolist()
df_classes['final_class'] = [limpeza_texto(class_name) for class_name in final_class]

df_classes

/dados01/workspace/ufmg.f01dcc/py37/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/dados01/workspace/ufmg.f01dcc/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


,real_meta-class,final_meta-class,processed_classes,count,final_class,validate,obs,cities,num_cities,example
0,EDITAL,EDITAL,edital licitacao,1207.0,edital,no,NaN,"passa-vinte, pirapetinga, coqueiral, sao-bento...",8.0,NaN
1,OUTROS,OUTROS,contrato administrativo,459.0,contrato,no,NaN,"coqueiral, pirapetinga",2.0,NaN
2,ATA,ATA,ata registro precos,345.0,ata registro precos,no,NaN,"coqueiral, pirapetinga, sao-bento-abade, cristais",4.0,NaN
3,EDITAL,EDITAL,edital pregao presencial,341.0,edital,no,NaN,"ribeirao-vermelho, pirapetinga, coqueiral, sao...",5.0,NaN
4,EDITAL,EDITAL,edital,277.0,edital,no,NaN,"passa-vinte, ribeirao-vermelho, pirapetinga, c...",6.0,NaN
...,...,...,...,...,...,...,...,...,...,...
270,OUTROS,OUTROS,cronograma financeiro,1.0,outros,no,NaN,olaria,1.0,79a70b2e386a7ae63c8435c2a41f12a6
271,OUTROS,OUTROS,custo secretaria,1.0,outros,no,São planilhas,arantina,1.0,d15742de0065437aa2223226aa014703
272,OUTROS,OUTROS,custos totais item,1.0,outros,no,São planilhas,coqueiral,1.0,7e25efd5d0788cd6b03ff60f357f3438
273,OUTROS,OUTROS,decisao impugnacao,1.0,impugnacao,no,NaN,coqueiral,1.0,a077d756699e6b653b6814d2bcf05b78


In [22]:
df_classes["final_class"].nunique()

49

In [23]:
df_classes.groupby('validate').size()

validate
no     235
yes     39
dtype: int64

In [24]:
df_classes_v1 = df_classes.groupby('processed_classes').agg({'final_class': lambda x: x.iloc[0], 'count':sum, 'cities': lambda x: list(x)}).reset_index()
df_classes_v1['cities'] = df_classes_v1['cities'].apply(lambda x: flatten([y.split(',') for y in x]))
df_classes_v1['num_cities'] = df_classes_v1['cities'].apply(lambda x: len(x))

df_classes_v1.to_csv('resultado_m03_meta_classes_extraction/anexo_a.csv')

### Gera Tabela 1

In [25]:
df_classes_v2 = df_classes.groupby('final_class').agg({'count':sum, 'cities': lambda x: list(x), 'final_meta-class': lambda x: list(x)}).reset_index()
df_classes_v2['cities'] = df_classes_v2['cities'].apply(lambda x: flatten([y.split(',') for y in x]))
df_classes_v2['final_meta-class'] = df_classes_v2['final_meta-class'].apply(lambda x: flatten([y.split(',') for y in x]))
df_classes_v2['num_cities'] = df_classes_v2['cities'].apply(lambda x: len(x))
df_classes_v2.to_csv('resultado_m03_meta_classes_extraction/tabela1.csv')
#df_classes_v2 = df_classes.groupby('processed_classes').agg({'count':sum, 'cities': lambda x: x.str.split(',')}).reset_index()
df_classes_v2

,final_class,count,cities,final_meta-class,num_cities
0,adendo,1.0,[sao-bento-abade],[OUTROS],1
1,adendo edital,23.0,"[cristais, sao-bento-abade]",[OUTROS],2
2,aditamento,212.0,[coqueiral],[OUTROS],1
3,adjudicacao,61.0,"[passa-vinte, pirapetinga]",[HOMOLOG],2
4,anexo,4.0,"[cristais, coqueiral, sao-bento-abade]",[OUTROS],3
5,anulacao licitacao,2.0,[pirapetinga],[OUTROS],1
6,ata dispensa licitacao,181.0,[cristais],[ATA],1
7,ata julgamento,11.0,"[sao-bento-abade, pirapetinga, cristais]",[ATA],3
8,ata pregao presencial,153.0,"[cristais, sao-bento-abade]",[ATA],2
9,ata registro precos,352.0,"[cristais, coqueiral, pirapetinga, sao-bento-a...",[ATA],4


# Gera tabela com classe final por documento

In [28]:
df_classe_final = pd.read_excel(file_dir, sheet_name="tabelaAgrupamento")
# Seleciona apenas colunas de classes
df_classe_final = df_classe_final[['final_class', 'Proposta de agrupamento']]
# Renomeia colunas
df_classe_final.columns = ['final_class', 'label']
# Recupera processed classes
df_classe_final = pd.merge(left = df_classe_final, right=df_classes[['real_meta-class','final_meta-class','processed_classes', 'final_class']], on='final_class')
# Recupera doc_id
df_classe_final = pd.merge(left = df_classe_final, right=df_rotulos[['doc_id', 'real_meta-class', 'processed_classes']], on = ['real_meta-class', 'processed_classes'])
# Filtra colunas relevantes
df_classe_final = df_classe_final[['doc_id', 'final_meta-class', 'label']]

display(df_classe_final.head())

df_classe_final.to_csv("resultado_m03_meta_classes_extraction/relacao_documentos_label.csv", index=False)

,doc_id,final_meta-class,label
0,499fa518c7724a6e449c509b7d7bc819,ATA,ata dispensa licitacao
1,56a09c5d1d04cc95ada4d68ad22dcbd9,ATA,ata dispensa licitacao
2,6e9f35208290c1130fcccad47b50aa53,ATA,ata dispensa licitacao
3,669c029c5812ec9a31b9c211044157b4,ATA,ata dispensa licitacao
4,a22421dc45d623c9200c7e8fb1e6ca34,ATA,ata dispensa licitacao


# Gera tabela com classe final agrupadas por documento

In [29]:
df_classe_final = pd.read_excel(file_dir, sheet_name="tabelaAgrupamento")
# Seleciona apenas colunas de classes
df_classe_final = df_classe_final[['final_class', 'Proposta de agrupamento']]
# Renomeia colunas
df_classe_final.columns = ['final_class', 'label']
# Recupera processed classes
df_classe_final = pd.merge(left = df_classe_final, right=df_classes[['real_meta-class','final_meta-class','processed_classes', 'final_class']], on='final_class')
# Recupera doc_id
df_classe_final = pd.merge(left = df_classe_final, right=df_rotulos[['doc_id', 'real_meta-class', 'processed_classes']], on = ['real_meta-class', 'processed_classes'])

# Agrupa classes de ata
atas_agrupar = ['ata julgamento', 'ata sessao publica']
df_classe_final.loc[df_classe_final['label'].isin(atas_agrupar), 'label'] = 'outras atas'

# Agrupa classes de homologacao e adjudicacao
adj_homo_agrupar = ['adjudicacao', 'homologacao']
df_classe_final.loc[df_classe_final['label'].isin(adj_homo_agrupar), 'label'] = 'homologacao adjudicacao'

# Agrupa outras classes 
outros_agrupar = ['outros termos', 'decisao/impugnacao', 'planilhas', 'memorial', 'extrato']
df_classe_final.loc[df_classe_final['label'].isin(outros_agrupar), 'label'] = 'outros'
# Filtra colunas relevantes
df_classe_final = df_classe_final[['doc_id', 'final_meta-class', 'label']]
df_classe_final.to_csv("resultado_m03_meta_classes_extraction/relacao_documentos_label_v2.csv", index=False)

In [ ]:
df_classe_final.groupby('label').size()